# Code pour prédictions en lot sur les données journalières sur modèle entrainé

In [1]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
from time import time
import itertools

import matplotlib.pyplot as plt
import collections


#Chargement des données à prédire depuis BQ dans un dataframe
from google.cloud import bigquery
client = bigquery.Client()                     #préparation du client pour interroger BigQuery et garder les data dans un pd.DataFrame

print('Querying BigQuery for daily shortage prediction analysis...')

sql_pred = """
SELECT DISTINCT * FROM `electric-armor-213817.Donnees_journalieres.Mise_en_Forme_Extract_journalier_MSTR_CSL`
WHERE PREPARATION_DATE < CURRENT_DATE
"""

start_time = time()

data_to_predict = client.query(sql_pred).to_dataframe()         #Interrogation de BigQuery 

print('Querying and loading time = {:0.2f} s '.format(time() - start_time))
print('Request finished\n')



print('Preprocessing input data...')



# ------------------ Load Training data to preprocess data ------------------------------------
sql_train = """
WITH
  DMS AS (
  SELECT
    ROW_NUMBER() OVER(PARTITION BY date_pr__pa, Code_SAP ORDER BY FLAG_CF ASC) AS ROW,
    FLAG_CF,
    CAST(date_pr__pa AS DATE) AS DATE_PREPA,
    Code_SAP,
    Import_type,
    Usine,
    Base,
    Index_Pareto,
    Cause_Pareto
  FROM
    `electric-armor-213817.Archives_DMS.DMS_2019_07_30`
  GROUP BY
    FLAG_CF,
    date_pr__pa,
    Code_SAP,
    Usine,
    Import_type,
    Base,
    Index_Pareto,
    Cause_Pareto )
  #----------------------------------------------------------------------------
SELECT DISTINCT
  FLAG_CF,
  PREPARATION_DATE,
  PDT_COD,
  PDT_DSC,
IF
  (Forecast_table.Transformable = 'OUI',
    1,
    0) AS TRANSFORMABLE,
  IF(DATE_DIFF( PREPARATION_DATE,CAST(BDD_PDT.Product_launch_date AS DATE), MONTH) < 2 ,1,0) AS INNO,
  IF(Reft = '01 Afh', 1,0) AS IMPLUSE,
  BDD_PDT.Umbrella_Brand,
  DMS.Import_type,
  DMS.Usine,
  PLT_COD,
  PLT_DSC,
  SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code ) AS PREVISION_NATIONAL,
  COMMANDE_NATIONAL,
  ALLOUE_NATIONAL,
  LIVRE_NATIONAL,
  (ALLOUE_NATIONAL - LIVRE_NATIONAL) AS ECART_ALLOC_NATIONAL,
  RUPTURE_NATIONAL,
  RUPTURE_STKA_NATIONAL,
IF
  (SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code ) = 0,
    1,
    0) AS NO_FORECAST_NAT,
  COMMANDE_NATIONAL - SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code ) AS ECART_PREVISION_NAT,
IF
  (SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code ) = 0,
    90000,
    -(SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code ) - COMMANDE_NATIONAL)*100 / SUM(Forecast_table.Forecast_including_adjustments) OVER(PARTITION BY Forecast_table.date, SAP_code )) AS ECART_PREVISION_NAT_PERCENT,
  COUNT(case when RUPTURE > 0 then 1 else null end) OVER(PARTITION BY Forecast_table.date, SAP_code) AS NB_OF_AFFECTED_DC,
  Forecast_table.Forecast_including_adjustments AS PREVISION,
  COMMANDE,
  ALLOUE,
  LIVRE,
  (ALLOUE - LIVRE) AS ECART_ALLOC,
  RUPTURE,
  RUPTURE_STKA,
IF
  (Forecast_table.Forecast_including_adjustments = 0,
    1,
    0) AS NO_FORECAST,
  COMMANDE - Forecast_table.Forecast_including_adjustments AS ECART_PREVISON,
IF
  (Forecast_table.Forecast_including_adjustments = 0,
    10000,
    -(Forecast_table.Forecast_including_adjustments - COMMANDE)*100 / Forecast_table.Forecast_including_adjustments) AS ECART_PREVISION_PERCENT,
  CSL_ALLOC,
  CSL,
  CSL_ALLOC - CSL AS PERTE_CSL_vs_ALLOC,
  CSL_ALLOC_NATIONAL,
  CSL_NATIONAL,
  CSL_ALLOC_NATIONAL - CSL_NATIONAL AS PERTE_CSL_vs_ALLOC_NATIONAL,
  DMS.Cause_Pareto
FROM
  `electric-armor-213817.Archives_MicroStrategy.CSL_20181201_to_20190630_clean` AS MSTR_Archive_table,
  `electric-armor-213817.Data_Forecasts.Data_Forecasts_CAR` AS Forecast_table,
  `electric-armor-213817.Fichiers_produits.REFERENTIEL_PRODUITS_V2` AS BDD_PDT,
  DMS
WHERE
  MSTR_Archive_table.PREPARATION_DATE = Forecast_table.date
  AND MSTR_Archive_table.PLT_DSC = Forecast_table.DC
  AND MSTR_Archive_table.PDT_COD = Forecast_table.SAP_code
  AND MSTR_Archive_table.PDT_COD = BDD_PDT.Codification
  AND MSTR_Archive_table.PREPARATION_DATE = DMS.DATE_PREPA
  AND MSTR_Archive_table.PDT_COD = DMS.Code_SAP 
  AND IF(DMS.FLAG_CF = 0, TRUE, MSTR_Archive_table.PLT_DSC = DMS.Base)     #Condition sur quelle base joindre dans le cas ou j'ai une CF ou non (car top 10 donné sans la base)
"""


start_time = time()
data = client.query(sql_train).to_dataframe()
print('Querying and loading time of training data = {:0.2f} s \n'.format(time() - start_time))


Querying BigQuery for daily shortage prediction analysis...
Querying and loading time = 15.34 s 
Request finished

Preprocessing input data...
Querying and loading time of training data = 9.16 s 



In [2]:
#-------------------Trainingset One Hot Encode of categorical data----------------------
data = data.drop(['PREPARATION_DATE', 'PLT_COD', 'PDT_COD', 'PDT_DSC'], axis = 1)

DC = pd.get_dummies(data.PLT_DSC)
PLANT = pd.get_dummies(data.Usine)
#MARQUE = pd.get_dummies(data.Umbrella_Brand)

data = pd.concat([PLANT,DC,data], axis=1)
data = data.drop(['PLT_DSC','Usine','Umbrella_Brand'], axis=1) 

training_feature_names = list(data)[:-1]            #le nom des colonnes des features d'entrainement
training_target_name = list(data)[-1]                #le nom de la colonne des classes cibles

#Put te Class_Name at the end of the DataFrame columns
training_columns = list(np.sort(training_feature_names))      #on arrange les colonnes issues du set d'entrainement 

In [3]:
#--------------- Prediction data preprocessing -----------------------------------------
#On conserve le DataFrame appelé 'data_to_predict' auquel on viendra ajouté ensuite les prédictions de l'algo
#on crée un DataFrame intermédiaire appelé 'df_for_prediction' qui récupère les données journalières pour transformer les données catégoriques et ne va prendre que les colonnes du dataset d'entrainement


#One Hot Encode des données catégoriques (Usine, DC) pour 
DC_pred = pd.get_dummies(data_to_predict.PLT_DSC)
PLANT_pred = pd.get_dummies(data_to_predict.Usine)
#MARQUE = pd.get_dummies(data.Umbrella_Brand)

df_for_prediction = pd.concat([PLANT_pred,DC_pred,data_to_predict], axis=1)

df_for_prediction = df_for_prediction[training_columns]
df_for_prediction = df_for_prediction.dropna()

In [4]:
print('Training dataframe has ',data.shape[1]-1, ' features plus one target column')
print('Prediction dataframe has ',df_for_prediction.shape[1], ' features')

if data.shape[1]-1 != df_for_prediction.shape[1]:
    raise ValueError("Number of features between training set and prediction set is different")

    
#On veut prédire la classe de toutes les ruptures, on conserve donc les données dans une liste que l'on fera passer dasn l'algo
X_to_predict = df_for_prediction.iloc[:, :].values

print('Data Preprocessing finished')

Training dataframe has  60  features plus one target column
Prediction dataframe has  60  features
Data Preprocessing finished


In [5]:
print('Batch prediction of daily shortages...')

#---------------------------- Choose which model to load -------------------
#get the trained model
from joblib import load
model = load('trained_XGB_V2.joblib')



#Perform predictions and get probabilities
y_to_predict = model.predict(X_to_predict)
y_to_predict_proba = model.predict_proba(X_to_predict)

y_to_predict_proba = np.amax(y_to_predict_proba, axis=1)

Y_to_predict_df = pd.DataFrame( {"Class_Prediction":y_to_predict})
Y_to_predict_proba_df = pd.DataFrame( {"Class_Prediction_probability":y_to_predict_proba})

#Build the predicted Pandas DataFrame
Final_predicted_df = pd.concat([data_to_predict,Y_to_predict_df,Y_to_predict_proba_df], axis=1)

Final_predicted_df = Final_predicted_df.dropna()

print('Batch prediction finished')

Batch prediction of daily shortages...
Batch prediction finished


In [6]:
#------------------------ Retravail des prédictions -----------------------

def flag_rupture(row):
    #Fonction qui flag lorsqu'il n'y a pas de rupture
    val=0
    if row['RUPTURE'] > 0:
        val = 1
    else:
        val = 0
    return val

def flag_livraison(row):
    #fonction qui va passer la classe à "Livré" si RUPTURE = 0
    classe=row['Class_Prediction']
    if row['RUPTURE'] == 0:
        classe = 'Livré'
    else:
        pass
    return classe
        

def rejet_cause_prev(row):
    #fonction qui rejete de la cause prévisions une rupture qui présente Prev>Commande 
    #ou pour lequel l'écart de prévision national est nul
    #ou pour lequel le CSL_alloc_National est de 100%
    classe = row['Class_Prediction']
    proba = row['Class_Prediction_probability']
    if (((row['PREVISION'] > row['COMMANDE'])  | (row['CSL_ALLOC_NATIONAL'] == 100.0) | (row['ECART_PREVISION_NAT_PERCENT'] == 0.0)) & (classe == 'Previsions')):
        classe = "A Creuser"
        proba = 1
    else:
        pass
    return classe    #il faudrait aussi modifier la probabilité à 1 mais je ne sais pas encore comment le faire




#Application de ces règles au dataframe
Final_predicted_df['FLAG_RUPTURE'] = Final_predicted_df.apply(flag_rupture, axis=1)

Final_predicted_df['Class_Prediction'] = Final_predicted_df.apply(rejet_cause_prev, axis=1)

Final_predicted_df['Class_Prediction'] = Final_predicted_df.apply(flag_livraison, axis=1)

In [7]:
print('Export to BigQuery table...')
start_time = time()

#Export vers BigQuery
bigquery_dataset_name = 'electric-armor-213817.Donnees_journalieres'
bigquery_table_name = 'Classification_journaliere'

# Define BigQuery dataset and table
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)


# Create or overwrite the existing table if it exists
table_schema = bq.Schema.from_data(Final_predicted_df)
table.create(schema = table_schema, overwrite = True)

# Write the DataFrame to a BigQuery table
table.insert(Final_predicted_df)
print('BigQuery export finished. \nExporting process took {:0.2f}min'.format((time()-start_time)/60))

Export to BigQuery table...
BigQuery export finished. 
Exporting process took 2.23min
